<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/codecompletion_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# We could modify these paths to "stub" behavior for test/dev
workspaceDir = "/content"
GPTNeoXColabDirName = "GPT-NeoX-Colab"
GPTNeoXColabDir = f"{workspaceDir}/{GPTNeoXColabDirName}"

# Clone CodeXGLUE Repo

In [2]:
!git clone https://github.com/microsoft/CodeXGLUE.git &

Cloning into 'CodeXGLUE'...
remote: Enumerating objects: 3373, done.
remote: Counting objects: 100% (3372/3372), done.
remote: Compressing objects: 100% (1534/1534), done.
remote: Total 3373 (delta 1746), reused 3329 (delta 1733), pack-reused 1 (from 1)
Receiving objects: 100% (3373/3373), 213.15 MiB | 20.36 MiB/s, done.
Resolving deltas: 100% (1746/1746), done.
Updating files: 100% (400/400), done.


In [3]:
#@title Clone GPT-NeoX-Colab
%%time
%cd {workspaceDir}
# Don't use --depth 1 because that does not play nice with git-annex
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
%cd {GPTNeoXColabDir}
#%pip install --use-feature=fast-deps -q -r requirements_colab.txt
!cat requirements_colab.txt | xargs -n 1 -P 8 pip install --use-feature=fast-deps -q
%pip install --use-feature=fast-deps -q .
from dotenv import load_dotenv
import os
load_dotenv(f"{GPTNeoXColabDir}/.env")
import GPTNeoXColab
GPTNeoXColab.utils.colab.fetch_data("data/codecompletion/token_completion.tar.gz")
GPTNeoXColab.utils.colab.fetch_data("models/codecompletion.tar.gz")
#%cd {GPTNeoXColabDir}/data/codecompletion
#!tar -xzf token_completion.tar.gz
%cd {GPTNeoXColabDir}/models
!tar -xzf codecompletion.tar.gz

/content
Cloning into 'GPT-NeoX-Colab'...
remote: Enumerating objects: 1348, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 1348 (delta 181), reused 285 (delta 179), pack-reused 1052 (from 1)
Receiving objects: 100% (1348/1348), 15.06 MiB | 32.33 MiB/s, done.
Resolving deltas: 100% (758/758), done.
/content/GPT-NeoX-Colab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Using Byte-Pair Encoding Tokenizer

In [4]:
%cd {workspaceDir}
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json &
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt

/content
--2024-11-14 09:27:03--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.234.112, 52.216.50.144, 52.217.204.16, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.234.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘gpt2-vocab.json’

gpt2-vocab.json     100%[===================>]   1018K  --.-KB/s    in 0.1s    

2024-11-14 09:27:03 (10.4 MB/s) - ‘gpt2-vocab.json’ saved [1042301/1042301]

--2024-11-14 09:27:03--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.234.112, 52.216.50.144, 52.217.204.16, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.234.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘gpt2-merges.txt’

gpt2-merges.txt     100%[===================>] 44

In [ ]:
#!pip show datasets
#!pip install datasets==1.18.0
#!pip install hf-transfer
#!pip install lm-eval --upgrade

# HuggingFace Inference

In [9]:
from transformers import GPTNeoXForCausalLM, GPT2Tokenizer
import torch

%cd {workspaceDir}

# Initialize the tokenizer with your vocabulary and merge files
tokenizer = GPT2Tokenizer(vocab_file=f"{workspaceDir}/gpt2-vocab.json", merges_file=f"{workspaceDir}/gpt2-merges.txt")

# Load your model
model_path = f"{GPTNeoXColabDir}/models/codecompletion"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Prompt the user for input
input_text = """<s> import sys , os <EOL> import imp <EOL> from optparse import make_option <EOL> from django . conf import settings <EOL> from django"""

# Tokenize and prepare input
input_ids = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode the generated text
generated_text = tokenizer.decode(output[0].tolist())

# Function to replace special tokens with original representations
def replace_special_tokens(text):
    replacements = {
        "<EOL>": "\n",  # Replace with actual newline
        "<s>": "",
        "</s>": "",     # Remove end token
        "<STR_LIT>": "STR_LITERAL",  # Example replacement, adjust as necessary
        "<NUM_LIT>": "NUM_LITERAL",   # Example replacement, adjust as necessary
    }

    for token, replacement in replacements.items():
        text = text.replace(token, replacement)

    return text.strip()  # Strip leading/trailing whitespace

# Replace special tokens in the generated text
final_text = replace_special_tokens(generated_text)

# Print the final output
print("Generated text:", final_text)


/content
Generated text: import sys , os 
 import imp 
 from optparse import make_option 
 from django . conf import settings 
 from django . utils . importlib import import_module 
 from django . core . management import call_command 
 from django . core . management import BaseCommand 
 from django . db import connections 
 def import_app ( app_label , verbosity ) : 
 try : 
 app_path = __import__ ( app_label , { } , { } , [ app_label . split ( '<STR_LIT:.>' ) [ - <NUM_LIT:1> ] ] ) . __path__ 
' , 
' 
' 
STR_LITERAL def0


In [ ]:
!cp {workspaceDir}/gpt2-vocab.json {GPTNeoXColabDir}/models/codecompletion/vocab.json
!cp {workspaceDir}/gpt2-merges.txt {GPTNeoXColabDir}/models/codecompletion/merges.txt
!cp {GPTNeoXColabDir}/data/codecompletion/token_completion.tar.gz {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150
%cd {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150
!tar -xzf token_completion.tar.gz

In [ ]:

%cd {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/code
!python -u run_lm.py \
        --data_dir=../dataset/py150/token_completion \
        --lit_file=../dataset/py150/literals.json \
        --langs=$LANG \
        --output_dir=../dataset/py150 \
        --pretrain_dir=/content/gpt-neox/hf_model/save/location \
        --log_file=../completion_python_eval.log \
        --model_type=gpt2 \
        --block_size=2048 \
        --do_eval \
        --per_gpu_eval_batch_size=4 \
        --logging_steps=100 \
        --seed=42